# Simulated_Echo_SVC
## Claude de Rijke-Thomas
### 15th March 2019

Open in tensorflowenv: source activate tensorflowenv

In [2]:
import scipy.io
import numpy as np
from scipy.integrate import simps, trapz
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, adam
from sklearn.neural_network import MLPClassifier
from numba import jit, prange
import pickle

/anaconda3/envs/tensorflowenv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


NLP1 corresponds to:
$$h\_s = [0, 0.05, 0.10, 0.15] $$
$$l\_surf = [0:1:10]; % large-scale correlation length (default = 5 m) $$
$$sigma\_si = [0.001:0.0005:0.004]; % sea ice rms height (default = 0.002 m) $$
$$sigma\_surf = [0:0.05:0.5]; % large-scale rms roughness height (default = 0.1 m)$$
NLP2 corresponds to:
$$h\_s =  [0.20, 0.25, 0.30, 0.35]$$
$$l\_surf = [0:1:10]; % large-scale correlation length (default = 5 m) $$
$$sigma\_si = [0.001:0.0005:0.004]; % sea ice rms height (default = 0.002 m) $$
$$sigma\_surf = [0:0.05:0.5]; % large-scale rms roughness height (default = 0.1 m)$$
NLP3 corresponds to:
$$h\_s =  [0.40, 0.45, 0.50]$$
$$l\_surf = [0:1:10]; % large-scale correlation length (default = 5 m) $$
$$sigma\_si = [0.001:0.0005:0.004]; % sea ice rms height (default = 0.002 m) $$
$$sigma\_surf = [0:0.05:0.5]; % large-scale rms roughness height (default = 0.1 m)$$

In [4]:
#Had to split the non-lead echo simulations into three parts: NLP1, NLP2 and NLP3
NLP1 = scipy.io.loadmat('/Users/claudederijkethomas/Desktop/Ice/Proj/FEM_Sim_NLP1.mat')
NLP2 = scipy.io.loadmat('/Users/claudederijkethomas/Desktop/Ice/Proj/FEM_Sim_NLP2.mat')
NLP3 = scipy.io.loadmat('/Users/claudederijkethomas/Desktop/Ice/Proj/FEM_Sim_NLP3.mat')

sim_params = NLP1['parameters_lookup']
sim_params_2 = NLP2['parameters_lookup']
sim_params_3 = NLP3['parameters_lookup']

sim_echoes = NLP1['P_t_ml_range']
sim_echoes_2 = NLP2['P_t_ml_range']
sim_echoes_3 = NLP3['P_t_ml_range']


In [5]:
def extractNLP1():
    counter = 0
    data_arr = []
    label_arr = []
    int_label_arr = []
    for i in range(len(sim_echoes[:][:][:][:])): #snow depth{s} (h_s)
        for j in range(len(sim_echoes[0][:][:][:])): #l_surf{s}
            for k in range(len(sim_echoes[0][0][:][:])): #sigma_si{s}
                for l in range(len(sim_echoes[0][0][0][:])): #sigma_surf{s}
                    simulated_echo = [sim_echoes[i][j][k][l][m][0] for m in range(len(sim_echoes[i][j][k][l]))]
                    sim_echo_params = [sim_params[i][j][k][l][0][m] for m in range(len(sim_params[i][j][k][l][0]))]
                    simulated_echo_area = trapz(simulated_echo, dx = 1) 
                    norm_sim_echo = [x/simulated_echo_area for x in simulated_echo]
                    data_arr.append(norm_sim_echo)
                    label_arr.append(str(sim_echo_params))
                    int_label_arr.append(counter)#+j*range(len(sim_echoes[0][:][:][:])) k*range(len(sim_echoes[0][0][:][:]))+l*range(len(sim_echoes[0][0][0][:]))))
                    counter+=1
    for i in range(len(sim_echoes_2[:][:][:][:])): #snow depth{s} (h_s)
        for j in range(len(sim_echoes_2[0][:][:][:])): #l_surf{s}
            for k in range(len(sim_echoes_2[0][0][:][:])): #sigma_si{s}
                for l in range(len(sim_echoes_2[0][0][0][:])): #sigma_surf{s}
                    simulated_echo = [sim_echoes_2[i][j][k][l][m][0] for m in range(len(sim_echoes_2[i][j][k][l]))]
                    sim_echo_params = [sim_params_2[i][j][k][l][0][m] for m in range(len(sim_params_2[i][j][k][l][0]))]
                    simulated_echo_area = trapz(simulated_echo, dx = 1) 
                    norm_sim_echo = [x/simulated_echo_area for x in simulated_echo]
                    data_arr.append(norm_sim_echo)
                    label_arr.append(str(sim_echo_params))
                    int_label_arr.append(counter)#+j*range(len(sim_echoes[0][:][:][:])) k*range(len(sim_echoes[0][0][:][:]))+l*range(len(sim_echoes[0][0][0][:]))))
                    counter+=1       
    for i in range(len(sim_echoes_3[:][:][:][:])): #snow depth{s} (h_s)
        for j in range(len(sim_echoes_3[0][:][:][:])): #l_surf{s}
            for k in range(len(sim_echoes_3[0][0][:][:])): #sigma_si{s}
                for l in range(len(sim_echoes_3[0][0][0][:])): #sigma_surf{s}
                    simulated_echo = [sim_echoes_3[i][j][k][l][m][0] for m in range(len(sim_echoes_3[i][j][k][l]))]
                    sim_echo_params = [sim_params_3[i][j][k][l][0][m] for m in range(len(sim_params_3[i][j][k][l][0]))]
                    simulated_echo_area = trapz(simulated_echo, dx = 1) 
                    norm_sim_echo = [x/simulated_echo_area for x in simulated_echo]
                    data_arr.append(norm_sim_echo)
                    label_arr.append(str(sim_echo_params))
                    int_label_arr.append(counter)#+j*range(len(sim_echoes[0][:][:][:])) k*range(len(sim_echoes[0][0][:][:]))+l*range(len(sim_echoes[0][0][0][:]))))
                    counter+=1                

    return np.array(data_arr), np.array(label_arr),np.array(int_label_arr)
feature_arr,label_arr,int_label_arr = extractNLP1() #for features for both the SVC and neural network, labels for the SVC and for labels for the neural network

In [6]:
print(np.shape(feature_arr)) #for features for both the SVC and neural network
print(np.shape(label_arr)) #for labels for the SVC
label_arr = np.array(label_arr, dtype = 'str')
print(np.shape(int_label_arr)) #for labels for the neural network

(9317, 139)
(9317,)
(9317,)


In [7]:
clf = SVC(gamma='auto') #creating the suppport vector classification
clf.fit(feature_arr, label_arr) #training the SVC using the simulated echoes and their corresponding parameters

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [8]:
#Adding NLP3 to the model as well: (total echoes = 5929) (another 2591 echoes):
def proportion_correct_predictions():
    correct_counter = 0
    incorrect_counter = 0
    for i in prange(np.shape(feature_arr)[0]):
        if i%20 ==0:
            #extracting an echo
            echo = feature_arr[i]
            # slightly modifying the echo (for subsequent denoising accuracy calculation)
            echo_mod = [np.random.normal(loc = echo[j], scale = echo[j]*0.05)  if j%5==0 else echo[j] for j in range(len(echo))]
            #checking if the modified echo is accurately predicted as the clean echo:
            if clf.predict([echo])[0] == clf.predict([echo_mod])[0]:
                #counting the number of modified echoes correctly denoised:
                correct_counter+=1
            else:
                #counting the number of modified echoes incorrectly denoised:
                incorrect_counter+=1
    #returning the proportion of correct predictions:
    return correct_counter/(incorrect_counter+correct_counter)
proportion_correct_predictions()

0.7360515021459227

In [ ]:
#Adding NLP3 to the model as well: (total echoes = 5929) (another 2591 echoes):
def proportion_correct_predictions():
    correct_counter = 0
    incorrect_counter = 0
    for i in prange(np.shape(feature_arr)[0]):
        
        #extracting an echo
        echo = feature_arr[i]
        # slightly modifying the echo (for subsequent denoising accuracy calculation)
        echo_mod = [np.random.normal(loc = echo[j], scale = echo[j]*0.025) for j in range(len(echo))]
        #checking if the modified echo is accurately predicted as the clean echo:
        if clf.predict([echo])[0] == clf.predict([echo_mod])[0]:
            #counting the number of modified echoes correctly denoised:
            correct_counter+=1
        else:
            #counting the number of modified echoes incorrectly denoised:
            incorrect_counter+=1
    #returning the proportion of correct predictions:
    return correct_counter/(incorrect_counter+correct_counter)
proportion_correct_predictions()

In [ ]:
echo = feature_arr[1000]
echo_mod = [np.random.normal(loc = echo[j], scale = echo[j]*0.025) for j in range(len(echo))]
plt.plot(echo, label = 'original echo')
plt.plot(echo_mod, label = 'corrupted echo')
plt.legend(loc = 'best')
plt.ylabel('Normalised Power', fontsize = 15)
plt.xlabel('range bin number', fontsize = 15)
plt.savefig("EchoCorruptionErr2.5.png", dpi = 200)

# The tease

In [ ]:
#Loading a CryoSat-2 echo from extracting_waveforms.ipynb file:
pickle_in = open("file.pickle", 'rb')
obs_arr = pickle.load(pickle_in)
clf.predict([obs_arr])
# And so the random CryoSat-2 echo is predicted to have a snow depth of 5cm, large-scale correlation length of 8cm, 
#  a sea ice rms height of 0.0015m and a large-scale rms roughness height of 0.5m. This is promising, as these values
#  lie close to the default values when compared to the range in values of the parameters that the SVC trained for,
# apart from sigma_surf, which lies on the edge of the parameter space. This has to be taken with some speculation however,
#  as it has been assumed that this echo doesnt contain a lead.


# The Lead Part

In [ ]:
LP1 = scipy.io.loadmat('/Users/claudederijkethomas/Desktop/Ice/Proj/FEM_Simulations_temp_Claude.mat')
sim_params = LP1['parameters_lookup']
sim_echoes = LP1['P_t_ml_range']
print(sim_echoes)

In [ ]:
def extractLP1():
    counter = 0
    data_arr = []
    label_arr = []
    int_label_arr = []
    for i in range(len(sim_echoes[:][:][:][:])): #snow depth{s} (h_s)
        for j in range(len(sim_echoes[0][:][:][:])): #l_surf{s}
            for k in range(len(sim_echoes[0][0][:][:])): #sigma_si{s}
                for l in range(len(sim_echoes[0][0][0][:])): #sigma_surf{s}
                    simulated_echo = [sim_echoes[i][j][k][l][m][0] for m in range(len(sim_echoes[i][j][k][l]))]
                    sim_echo_params = [sim_params[i][j][k][l][0][m] for m in range(len(sim_params[i][j][k][l][0]))]
                    simulated_echo_area = trapz(simulated_echo, dx = 1) 
                    norm_sim_echo = [x/simulated_echo_area for x in simulated_echo]
                    data_arr.append(norm_sim_echo)
                    label_arr.append(str(sim_echo_params))
                    int_label_arr.append(counter)#+j*range(len(sim_echoes[0][:][:][:])) k*range(len(sim_echoes[0][0][:][:]))+l*range(len(sim_echoes[0][0][0][:]))))
                    counter+=1
    return np.array(data_arr), np.array(label_arr),np.array(int_label_arr)
feature_arr,label_arr,int_label_arr = extractLP1() #for features for both the SVC and neural network, labels for the SVC and for labels for the neural network

In [ ]:
print(np.shape(feature_arr))
print(np.shape(label_arr))

In [ ]:
label_arr = np.array(label_arr, dtype = 'str')
clf = SVC(gamma='auto') #creating the suppport vector classification
clf.fit(feature_arr, label_arr) #training the SVC using the simulated echoes and their corresponding parameters

In [ ]:
def proportion_correct_predictions():
    correct_counter = 0
    incorrect_counter = 0
    for i in prange(np.shape(feature_arr)[0]):
        
        #extracting an echo
        echo = feature_arr[i]
        # slightly modifying the echo (for subsequent denoising accuracy calculation)
        echo_mod = [np.random.normal(loc = echo[j], scale = echo[j]*0.05)  if j%5==0 else echo[j] for j in range(len(echo))]
        #checking if the modified echo is accurately predicted as the clean echo:
        if clf.predict([echo])[0] == clf.predict([echo_mod])[0]:
            #counting the number of modified echoes correctly denoised:
            correct_counter+=1
        else:
            #counting the number of modified echoes incorrectly denoised:
            incorrect_counter+=1
    #returning the proportion of correct predictions:
    return correct_counter/(incorrect_counter+correct_counter)
proportion_correct_predictions()

In [ ]:
@jit
def proportion_correct_predictions():
    correct_counter = 0
    incorrect_counter = 0
    for i in prange(np.shape(feature_arr)[0]):
        
        #extracting an echo
        echo = feature_arr[i]
        # slightly modifying the echo (for subsequent denoising accuracy calculation)
        echo_mod =[]
        for j in prange(len(echo)):
            if j%5==0:
                np.append(echo_mod, np.random.normal(loc = echo[j], scale = echo[j]*0.05))
            else:
                np.append(echo_mod, echo[j])

        #checking if the modified echo is accurately predicted as the clean echo:
        if clf.predict([echo])[0] == clf.predict([echo_mod])[0]:
            #counting the number of modified echoes correctly denoised:
            correct_counter+=1
        else:
            #counting the number of modified echoes incorrectly denoised:
            incorrect_counter+=1
    #returning the proportion of correct predictions:
    return correct_counter/(incorrect_counter+correct_counter)
proportion_correct_predictions()

# The Neural Network (Failed)

In [ ]:
one_hot_labels = to_categorical(int_label_arr)

In [ ]:
def create_model():
    model = Sequential()
    #adding 400 nodes to the shallow layer:
    model.add(Dense(400, input_dim=int(139), kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.2))
    # Another hidden layer of 16 units
    model.add(Dense(400, kernel_initializer='normal', activation='relu'))
    #output layer
    model.add(Dense(3388, kernel_initializer='normal', activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
# Wrap our Keras model in an estimator compatible with scikit_learn
estimator = KerasClassifier(build_fn=create_model, epochs=30, verbose=0)
# Now we can use scikit_learn's cross_val_score to evaluate this model identically to the others
cv_scores = cross_val_score(estimator, feature_arr, one_hot_labels, cv=15) # cv = train test split (k-1: 1)
cv_scores.mean()

Taking another approach

In [ ]:
clfNN = MLPClassifier(solver='adam', alpha=1e-2, hidden_layer_sizes=(400, 16), random_state=1)
clfNN.fit(feature_arr, one_hot_labels)

In [ ]:
print(np.max(np.array(clfNN.predict([last_echo])[0])))